In [ ]:
def NNMF(dataset,layers,day,mouse,timeframes = 3):
    trainS = dataset["S"][0][day]
    opto = dataset["opt"][0][day]

    layers.insert(0,trainS.shape[0]*timeframes)

    trainData = []
    for i in range(trainS.shape[1]-timeframes+1):
        segment = trainS[:,i:i+timeframes]
        flatt = segment.flatten(order='F')
        trainData.append(flatt)
    trainMatrix = np.array(trainData).T

    train__ = torch.tensor(trainMatrix)
    net = Neural_NMF(layers)
    history = train(net, train__, epoch=10, lr=500, supervised=False)

    A = history[0][len(layers)-2]
    S = history[1][len(layers)-2]
    W1 = history[0][0].detach().numpy()
    W = A.detach().numpy()
    H = S.detach().numpy()

    # count_ones = np.count_nonzero(opto == 1)
    opto = opto.flatten()
    opto_extra = opto
    opto_extra[1:] = np.diff(opto) == 1
    opto_extra[0] = opto[0]  # Retain the
    indices = np.where(opto_extra == 1)[0]
    slices = []
    for idx in indices:
        if idx + 20 <= H.shape[1]:
            slices.append(H[:, idx:idx+20])
        else:
            slices.append(H[:, idx:])
    H_segment = np.hstack(slices) if slices else np.array([])


    plt.imshow(W1[:,:], cmap='coolwarm', interpolation='nearest')
    plt.colorbar()  # Show color scale
    plt.title(r'Heat map of $W^{(1)}$, ' f'for Session {day+1}')
    plt.show()

    plt.imshow(W[:,:], cmap='coolwarm', interpolation='nearest')
    plt.colorbar()  # Show color scale
    plt.title(r'Heat map of $W^{(2)}$, ' f'for Session {day+1}')
    plt.show()

    plt.imshow(H_segment[:,:100], cmap='coolwarm', interpolation='nearest')
    plt.colorbar()  # Show color scale
    plt.title(r'Heat map of H ' f'matrix for Session {day+1}')
    plt.show()

    #activations of neurons

    number = layers[1]
    y = np.arange(1, number+1)
    # Create the horizontal stacked bar graph
    fig, ax = plt.subplots(figsize=(5, number*0.1))  # Adjust the size to fit the labels better
    bar_height = 0.35
    index = np.arange(number)
    # Plot the bars
    bar1 = ax.barh(index, W[:number, 0], bar_height, label='Column 1')
    bar2 = ax.barh(index, W[:number, 1], bar_height, left=W[:number, 0], label='Column 2')
    # Adding labels and title
    ax.set_ylabel('Neurons')
    ax.set_xlabel('Activation Value')
    ax.set_title(r'Activation of Each Neuron in $W^{(2)}$ 'f'matrix for Session {day+1}')
    ax.set_yticks(index)
    ax.legend()
    plt.show()

    S2 = (A@S).detach().numpy()
    slices = []
    for idx in indices:
        if idx + 20 <= S2.shape[1]:
            slices.append(S2[:, idx:idx+10])
        else:
            slices.append(S2[:, idx:])
    D = np.hstack(slices) if slices else np.array([])

    plt.imshow(D[:,0:100], cmap='hot', interpolation='nearest')
    plt.title(r'Heat map of $H^{(1)}$ ' f'matrix for Session {day+1}')
    plt.show()

#     threshold = 0.07  # Adjust this as needed (e.g., test multiple values from 0 to 1)

# # Find stimulation indices

#     # Calculate precision for each row
#     precisions = []
#     for i, row in enumerate(H):
#         # Total motifs (values exceeding the threshold)
#         total_motifs_indices = np.where(row > threshold)[0]
        
#         # Motifs during stimulation (intersection of stimulated indices and motif indices)
#         stimulated_motifs_indices = np.intersect1d(stimulation_indices, total_motifs_indices)
        
#         # Calculate precision
#         precision = len(stimulated_motifs_indices) / len(total_motifs_indices) if len(total_motifs_indices) > 0 else 0
#         precisions.append(precision)
#         print(f"Row {i}: Precision = {precision:.3f}")

#     # Visualize precision across rows


#     plt.figure(figsize=(8, 6))
#     plt.bar(range(len(precisions)), precisions, color='skyblue')
#     plt.title('Precision Across Rows')
#     plt.xlabel('Row Index')
#     plt.ylabel('Precision')
#     plt.grid(axis='y')
#     plt.show()

    thresholds = np.linspace(0, 0.07, 50)

    # Find stimulation indices
    stimulation_indices = indices
    row_precisions = []
    row_recalls = []
    row_f1_scores = []

    # Calculate precision, recall, and F1 score for each threshold and each row
    for i, row in enumerate(H):
        precisions_for_row = []
        recalls_for_row = []
        f1_scores_for_row = []
        
        for threshold in thresholds:
            total_motifs_indices = np.where(row > threshold)[0]  # All motif indices
            stimulated_motifs_count = 0

            for stim_idx in stimulation_indices:
                # Check for values above threshold within the next 10 columns
                nearby_indices = total_motifs_indices[
                    (total_motifs_indices >= stim_idx) & (total_motifs_indices < stim_idx + 10)
                ]
                if len(nearby_indices) > 0:  # If there's a motif nearby, count it
                    stimulated_motifs_count += 1

            # Calculate Precision
            if len(total_motifs_indices) > 0:
                precision = stimulated_motifs_count / len(total_motifs_indices)
            else:
                precision = 0
            precisions_for_row.append(precision)

            # Calculate Recall
            if len(stimulation_indices) > 0:
                recall = stimulated_motifs_count / len(stimulation_indices)
            else:
                recall = 0
            recalls_for_row.append(recall)

            # Calculate F1 Score
            if precision + recall > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)
            else:
                f1_score = 0
            f1_scores_for_row.append(f1_score)
        
        row_precisions.append(precisions_for_row)
        row_recalls.append(recalls_for_row)
        row_f1_scores.append(f1_scores_for_row)

    # Convert lists to NumPy arrays
    row_precisions = np.array(row_precisions)
    row_recalls = np.array(row_recalls)
    row_f1_scores = np.array(row_f1_scores)

    # Filter thresholds and metrics to exclude regions where they become 0
    filtered_metrics = {
        "Precision": [],
        "Recall": [],
        "F1 Score": []
    }
    filtered_thresholds = []

    for i in range(H.shape[0]):
        for metric_name, metric_values in zip(["Precision", "Recall", "F1 Score"], 
                                            [row_precisions, row_recalls, row_f1_scores]):
            metric = metric_values[i]
            first_zero_idx = np.where(metric == 0)[0]
            if len(first_zero_idx) > 0:
                cutoff = first_zero_idx[0]  # Find the first index where the metric becomes 0
                filtered_metrics[metric_name].append(metric[:cutoff])
                if len(filtered_thresholds) <= i:
                    filtered_thresholds.append(thresholds[:cutoff])
            else:
                filtered_metrics[metric_name].append(metric)
                if len(filtered_thresholds) <= i:
                    filtered_thresholds.append(thresholds)

    # Plot filtered Precision, Recall, and F1 Score for each row
    for metric_name, metric_values in filtered_metrics.items():
        plt.figure(figsize=(10, 8))
        for i in range(H.shape[0]):
            plt.plot(filtered_thresholds[i], metric_values[i], label=f'Row {i}', marker='o')
        plt.title(f'{metric_name} vs Threshold for All Rows (Filtered)')
        plt.xlabel('Threshold')
        plt.ylabel(metric_name)
        plt.legend()
        plt.grid(True)
        plt.show()

    # Optional: Subplots for each row
    for metric_name, metric_values in filtered_metrics.items():
        fig, axes = plt.subplots(H.shape[0], 1, figsize=(8, 3 * H.shape[0]), sharex=True)
        fig.suptitle(f'{metric_name} vs Threshold (Filtered for Each Row)', fontsize=16)

        for i, ax in enumerate(axes):
            ax.plot(filtered_thresholds[i], metric_values[i], marker='o', label=f'Row {i}')
            ax.set_title(f'Row {i}')
            ax.set_ylabel(metric_name)
            ax.grid(True)
            ax.legend()

        # Add shared x-axis label
        plt.xlabel('Threshold')
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

    return history,W,H,H_segment